In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
# Entrena un modelo de Gradient Boosted Trees
# para predecir el consumo eléctrico
# y la cantidad de usuarios.

# Prueba diferentes modelos.

# Tomado y modificado a partir de:
# https://www.datacamp.com/community/tutorials/xgboost-in-python
# https://towardsdatascience.com/a-beginners-guide-to-xgboost-87f5d4c30ed7

import pandas as pd
import numpy as np

import xgboost as xgb

from joblib import dump

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [ ]:
path_drive = "/content/drive/MyDrive/Colab/Subsidio_electricidad/" 
path_data = path_drive + "data/Actual/data"
fname = ".csv"

test = []
reg_model = []

data = pd.read_csv(path_data + fname)
data_y = pd.read_csv(path_data + "_yearly" + fname)

# Unimos las tarifas 1 y DAC.
data["Consumo_1*"] += data["Consumo_DAC"]
data.drop(["Consumo_DAC", "Usuarios_1*", "Usuarios_DAC"],
    axis = 1, inplace = True)
data_y["Usuarios_1*"] += data_y["Usuarios_DAC"]
data_y.drop(["Consumo_1*", "Consumo_DAC", "Usuarios_DAC"],
    axis = 1, inplace = True)

# Escogemos las variables relevantes.
data.drop(["Area", "CVE_ENT", "CVE_MUN",
    "Tmax", "Tmin", "HDD_mean", "HDD_p10",
    "CDD_p90", "$luz", "Tmean_max_2",
    "M_verano", "Tarifa", "Verano"],
    axis = 1, inplace = True)

# Información utilizada para el modelo.
data.head()

,CVE_INEGI,NOM_ENT,NOM_MUN,Consumo_1*,lon,lat,Tmean,Pre,CDD_mean,Pre_Tmean,Poblacion,PIB,Densidad_poblacion,PCI,$GLP,Año,Mes
0,1001,Aguascalientes,Aguascalientes,2.297112e+07,-102.295872,21.8114,14.446821,15.308543,0.000000,0.000000,797010.0,1.170000e+11,6.833315,146798.659992,10.151,2010,1
1,1001,Aguascalientes,Aguascalientes,2.108122e+07,-102.295872,21.8114,14.861328,27.699915,0.000000,0.000000,797010.0,1.170000e+11,6.833315,146798.659992,10.151,2010,2
2,1001,Aguascalientes,Aguascalientes,2.377894e+07,-102.295872,21.8114,17.526365,4.332035,0.897792,0.976778,797010.0,1.170000e+11,6.833315,146798.659992,10.151,2010,3
3,1001,Aguascalientes,Aguascalientes,2.591731e+07,-102.295872,21.8114,19.526386,4.361156,4.786536,1.510271,797010.0,1.170000e+11,6.833315,146798.659992,10.151,2010,4
4,1001,Aguascalientes,Aguascalientes,2.676120e+07,-102.295872,21.8114,24.024788,10.045719,122.205048,10.045719,797010.0,1.170000e+11,6.833315,146798.659992,10.151,2010,5


In [ ]:
# Estadísticos de las variables.
data.describe()

,CVE_INEGI,Consumo_1*,lon,lat,Tmean,Pre,CDD_mean,Pre_Tmean,Poblacion,PIB,Densidad_poblacion,PCI,$GLP,Año,Mes
count,207396.000000,2.073960e+05,207396.000000,207396.000000,207396.000000,207396.000000,207396.000000,207396.000000,2.073960e+05,2.073960e+05,207396.000000,2.073960e+05,207396.000000,207396.000000,207396.000000
mean,19324.164844,1.796892e+06,-98.743039,20.022814,22.416142,92.261068,38.644583,61.727996,4.726342e+04,6.765280e+09,2.879387,8.464168e+04,10.993159,2013.000000,6.500000
std,7381.414984,7.318148e+06,4.384575,3.345745,4.394156,117.350002,47.328752,89.113685,1.360137e+05,2.694244e+10,11.833998,7.166228e+04,0.851131,2.000005,3.452061
min,1001.000000,1.859828e+03,-116.889018,14.639368,6.041615,0.000000,0.000000,0.000000,8.600000e+01,2.373494e+06,0.001441,1.273697e+04,8.948300,2010.000000,1.000000
25%,14079.000000,8.600011e+04,-100.737771,17.620009,19.377630,11.312651,3.530260,0.000616,4.273000e+03,2.402809e+08,0.191168,4.303295e+04,10.238682,2011.000000,3.750000
50%,20226.000000,2.677753e+05,-98.236805,19.329310,22.201066,43.194197,20.967391,20.387224,1.301700e+04,8.416350e+08,0.527785,6.653278e+04,11.104057,2013.000000,6.500000
75%,24027.000000,8.348410e+05,-96.769306,20.934726,25.490744,135.280884,58.762242,95.069677,3.362100e+04,2.806882e+09,1.374680,9.753169e+04,11.637004,2015.000000,9.250000
max,32058.000000,2.492331e+08,-86.908934,32.434882,36.112892,1161.472156,374.363434,983.411197,1.830284e+06,4.900000e+11,176.561449,1.234769e+06,13.169542,2016.000000,12.000000


In [ ]:
# Matriz de correlación de Pearson.
data.corr()

,CVE_INEGI,Consumo_1*,lon,lat,Tmean,Pre,CDD_mean,Pre_Tmean,Poblacion,PIB,Densidad_poblacion,PCI,$GLP,Año,Mes
CVE_INEGI,1.000000e+00,-0.064943,2.369749e-01,-1.816479e-02,0.133952,0.050134,-0.008833,0.062141,-1.299527e-01,-1.309971e-01,-7.796424e-02,-1.489635e-01,1.802796e-02,-1.711015e-19,-1.313476e-18
Consumo_1*,-6.494341e-02,1.000000,-1.491145e-01,1.932962e-01,0.051079,-0.040446,0.100556,-0.020965,7.938506e-01,7.365735e-01,3.130887e-01,2.760829e-01,5.806551e-03,1.292096e-02,1.080547e-02
lon,2.369749e-01,-0.149114,1.000000e+00,-6.648263e-01,0.283641,0.219681,-0.215081,0.141434,-1.116940e-01,-8.518956e-02,-2.431198e-04,-2.705908e-01,-1.311473e-02,2.656699e-19,1.810823e-20
lat,-1.816479e-02,0.193296,-6.648263e-01,1.000000e+00,-0.134246,-0.214621,0.357260,-0.128432,1.210389e-01,1.100495e-01,-2.975554e-02,3.687962e-01,3.705866e-02,1.433593e-19,6.080992e-20
Tmean,1.339519e-01,0.051079,2.836405e-01,-1.342458e-01,1.000000,0.292602,0.536167,0.355719,-5.115129e-02,-3.733268e-02,-1.342712e-01,-1.688623e-03,1.954647e-02,3.653135e-02,3.716337e-02
Pre,5.013443e-02,-0.040446,2.196809e-01,-2.146209e-01,0.292602,1.000000,0.055692,0.866616,-4.744698e-02,-4.332692e-02,-3.202255e-02,-1.159776e-01,-4.194580e-02,-2.717907e-02,2.215585e-01
CDD_mean,-8.832647e-03,0.100556,-2.150812e-01,3.572598e-01,0.536167,0.055692,1.000000,0.233260,3.219185e-02,3.117878e-02,-2.405607e-02,1.241233e-01,2.567613e-03,2.669760e-02,-5.177694e-02
Pre_Tmean,6.214123e-02,-0.020965,1.414337e-01,-1.284320e-01,0.355719,0.866616,0.233260,1.000000,-2.556043e-02,-2.647006e-02,-1.034226e-02,-6.723960e-02,-2.761061e-03,1.401004e-02,1.672546e-01
Poblacion,-1.299527e-01,0.793851,-1.116940e-01,1.210389e-01,-0.051151,-0.047447,0.032192,-0.025560,1.000000e+00,8.821186e-01,5.607455e-01,2.836590e-01,9.278487e-03,8.641860e-03,1.427132e-18
PIB,-1.309971e-01,0.736574,-8.518956e-02,1.100495e-01,-0.037333,-0.043327,0.031179,-0.026470,8.821186e-01,1.000000e+00,5.851398e-01,4.732125e-01,1.247851e-02,1.369315e-02,1.868866e-18


# Usuarios

In [ ]:
# Predicción de usuarios.

# Escogemos el conjunto de features y de variables a predecir.
Y = data.iloc[:,3]
X = data[["Poblacion", "PIB"]]


# Separamos en conjuntos de entrenamiento y de prueba.
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size = 0.3 )

# Separamos en conjunto de prueba y de validación.
X_train, X_val, Y_train, Y_val = train_test_split(
    X, Y, test_size = 0.5 )

In [ ]:
# Features a utilizar.
X.head()

,Poblacion,PIB
0,797010.0,1.170000e+11
1,797010.0,1.170000e+11
2,797010.0,1.170000e+11
3,797010.0,1.170000e+11
4,797010.0,1.170000e+11


In [ ]:
# Variable a predecir.
Y.head()

0    2.297112e+07
1    2.108122e+07
2    2.377894e+07
3    2.591731e+07
4    2.676120e+07
Name: Consumo_1*, dtype: float64

In [ ]:
# Regresión lineal.

# Copiamos las variables para poder
# transformarlas en caso de ser necesario.
X_reg = X_train.copy()
Y_reg = Y_train.copy()
X_reg_val = X_val.copy()
X_reg_test = X_test.copy()

# Creamos el regresor.
lin_reg = LinearRegression()

# Entrenamos el modelo.
lin_reg.fit(X_reg, Y_reg)

# Probamos la regresión en el set de entrenamiento.
preds = lin_reg.predict(X_reg)

# Calculamos el error de entrenamiento.
rmse = np.sqrt(mean_squared_error(Y_reg, preds))
r2 = r2_score(Y_reg, preds)

print("Entrenamiento")
print(f"RMSE: {rmse:.3E}")
print(f"R^2:  {r2:.7f}")

Entrenamiento
RMSE: 4.275E+06
R^2:  0.6402660


In [ ]:
# Probamos la regresión en el set de validación.
preds = lin_reg.predict(X_reg_val)

# Calculamos el error de validación.
rmse = np.sqrt(mean_squared_error(Y_val, preds))
r2 = r2_score(Y_val, preds)

print("Validación")
print(f"RMSE: {rmse:.3E}")
print(f"R^2:  {r2:.7f}")

# Probamos la regresión en el set de prueba
# y guardamos el resultado para su evaluación final.
test_0a = lin_reg.predict(X_reg_test)

Validación
RMSE: 4.551E+06
R^2:  0.6321210


In [ ]:
# Entrenamiento con transformación logarítmica.

# Copiamos las variables para poder
# transformarlas en caso de ser necesario.
X_reg = X_train.copy()
Y_reg = Y_train.copy()

X_reg_val = X_val.copy()
X_reg_test = X_test.copy()

# Sacamos el logaritmo.
X_reg[["Poblacion", "PIB"]] = np.log(
    X_reg[["Poblacion", "PIB"]])
Y_reg = np.log(Y_reg)
X_reg_val[["Poblacion", "PIB"]] = np.log(
    X_reg_val[["Poblacion", "PIB"]])
X_reg_test[["Poblacion", "PIB"]] = np.log(
    X_reg_test[["Poblacion", "PIB"]])

# Hiperparámetros.
params = {
    "objective": "reg:squarederror",
    "colsample_bytree": 0.3,
    "learning_rate": 0.1,
    "max_depth": 50,
    "min_child_weight" : 25,
    "min_split_loss": 0.3,
    "n_estimators": 100
    }

# Creamos el regresor con los hiperparámetros.
xg_reg = xgb.XGBRegressor( **params )

# Entrenamos el modelo.
xg_reg.fit(X_reg, Y_reg, verbose = True)

# Probamos la regresión en el set de entrenamiento.
preds = xg_reg.predict(X_reg)

# Quitamos el logaritmo para evaluar el error.
Y_reg = np.exp(Y_reg)
preds = np.exp(preds)

# Calculamos el error de entrenamiento.
rmse = np.sqrt(mean_squared_error(Y_reg, preds))
r2 = r2_score(Y_reg, preds)

print("Entrenamiento")
print(f"RMSE: {rmse:.3E}")
print(f"R^2:  {r2:.7f}")

Entrenamiento
RMSE: 2.128E+06
R^2:  0.9108469


In [ ]:
# Probamos la regresión en el set de validación.
preds = xg_reg.predict(X_reg_val)

# Quitamos el logaritmo para evaluar el error.
preds = np.exp(preds)

# Calculamos el error de validación.
rmse = np.sqrt(mean_squared_error(Y_val, preds))
r2 = r2_score(Y_val, preds)

print("Validación")
print(f"RMSE: {rmse:.3E}")
print(f"R^2:  {r2:.7f}")

# Probamos la regresión en el set de prueba
# y guardamos el resultado para su evaluación final.
test_0b = np.exp(xg_reg.predict(X_reg_test))

Validación
RMSE: 2.315E+06
R^2:  0.9048598


In [ ]:
# Calculamos el error de prueba.
rmse = np.sqrt(mean_squared_error(Y_test, test_0b))
r2 = r2_score(Y_test, test_0b)

print("Prueba")
print(f"RMSE: {rmse:.3E}")
print(f"R^2:  {r2:.7f}")

# Guardamos el modelo entrenado.
dump(lin_reg, path_drive + "reg_model_usuarios.joblib")

Prueba
RMSE: 2.112E+06
R^2:  0.9134087


['/content/drive/MyDrive/Colab/Subsidio_electricidad/reg_model_usuarios.joblib']

# Consumo

In [ ]:
# Predicción de consumo.
# Escogemos el conjunto de features y de variables a predecir.
X, Y = data.iloc[:,4:], data.iloc[:,3]

# Quitamos algunas features.
X.drop(["Pre", "PCI", "Densidad_poblacion"
    ],
    #, "Año"],
    axis = 1, inplace = True)

# Separamos en conjuntos de entrenamiento y de prueba.
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size = 0.3 )

# Separamos en conjunto de prueba y de validación.
X_train, X_val, Y_train, Y_val = train_test_split(
    X, Y, test_size = 0.5 )

In [ ]:
# Features a utilizar.
X.head()

,lon,lat,Tmean,CDD_mean,Pre_Tmean,Poblacion,PIB,$GLP,Año,Mes
0,-102.295872,21.8114,14.446821,0.000000,0.000000,797010.0,1.170000e+11,10.151,2010,1
1,-102.295872,21.8114,14.861328,0.000000,0.000000,797010.0,1.170000e+11,10.151,2010,2
2,-102.295872,21.8114,17.526365,0.897792,0.976778,797010.0,1.170000e+11,10.151,2010,3
3,-102.295872,21.8114,19.526386,4.786536,1.510271,797010.0,1.170000e+11,10.151,2010,4
4,-102.295872,21.8114,24.024788,122.205048,10.045719,797010.0,1.170000e+11,10.151,2010,5


In [ ]:
# Variable a predecir.
Y.head()

0    2.297112e+07
1    2.108122e+07
2    2.377894e+07
3    2.591731e+07
4    2.676120e+07
Name: Consumo_1*, dtype: float64

In [ ]:
# Método 1.
# Regresión lineal.

# Copiamos las variables para poder
# transformarlas en caso de ser necesario.
X_reg = X_train.copy()
Y_reg = Y_train.copy()
X_reg_val = X_val.copy()
X_reg_test = X_test.copy()

# Creamos el regresor.
lin_reg_1 = LinearRegression()

# Entrenamos el modelo.
lin_reg_1.fit(X_reg, Y_reg)

# Probamos la regresión en el set de entrenamiento.
preds = lin_reg_1.predict(X_reg)

# Calculamos el error de entrenamiento.
rmse = np.sqrt(mean_squared_error(Y_reg, preds))
r2 = r2_score(Y_reg, preds)

print("Entrenamiento")
print(f"RMSE: {rmse:.3E}")
print(f"R^2:  {r2:.7f}")

Entrenamiento
RMSE: 4.216E+06
R^2:  0.6632978


In [ ]:
# Probamos la regresión en el set de validación.
preds = lin_reg_1.predict(X_reg_val)

# Calculamos el error de validación.
rmse = np.sqrt(mean_squared_error(Y_val, preds))
r2 = r2_score(Y_val, preds)

print("Validación")
print(f"RMSE: {rmse:.3E}")
print(f"R^2:  {r2:.7f}")

# Probamos la regresión en el set de prueba
# y guardamos el resultado para su evaluación final.
test_1 = lin_reg_1.predict(X_reg_test)

test.append(test_1)
reg_model.append(lin_reg_1)

Validación
RMSE: 4.326E+06
R^2:  0.6555975


In [ ]:
# Método 2.
# Regresión lineal con transformación logarítmica.

# Copiamos las variables para poder
# transformarlas en caso de ser necesario.
X_reg = X.copy()
Y_reg = Y.copy()
X_reg_val = X_val.copy()
X_reg_test = X_test.copy()

# Sacamos el logaritmo.
X_reg[["Poblacion", "PIB"]] = np.log(
    X_reg[["Poblacion", "PIB"]])
Y_reg = np.log(Y_reg)
X_reg_val[["Poblacion", "PIB"]] = np.log(
    X_reg_val[["Poblacion", "PIB"]])
X_reg_test[["Poblacion", "PIB"]] = np.log(
    X_reg_test[["Poblacion", "PIB"]])

# Creamos el regresor.
lin_reg_2 = LinearRegression()

# Entrenamos el modelo.
lin_reg_2.fit(X_reg, Y_reg)

# Probamos la regresión en el set de entrenamiento.
preds = lin_reg_2.predict(X_reg)

# Quitamos el logaritmo para evaluar el error.
Y_reg = np.exp(Y_reg)
preds = np.exp(preds)

# Calculamos el error de entrenamiento.
rmse = np.sqrt(mean_squared_error(Y_reg, preds))
r2 = r2_score(Y_reg, preds)

print("Entrenamiento")
print(f"RMSE: {rmse:.3E}")
print(f"R^2:  {r2:.7f}")

Entrenamiento
RMSE: 3.476E+06
R^2:  0.7743349


In [ ]:
# Probamos la regresión en el set de validación.
preds = lin_reg_2.predict(X_reg_val)

# Quitamos el logaritmo para evaluar el error.
preds = np.exp(preds)

# Calculamos el error de validación.
rmse = np.sqrt(mean_squared_error(Y_val, preds))
r2 = r2_score(Y_val, preds)

print("Validación")
print(f"RMSE: {rmse:.3E}")
print(f"R^2:  {r2:.7f}")

# Probamos la regresión en el set de prueba
# y guardamos el resultado para su evaluación final.
test_2 = lin_reg_2.predict(X_reg_test)
test_2 = np.exp(test_2)

test.append(test_2)
reg_model.append(lin_reg_2)

Validación
RMSE: 3.502E+06
R^2:  0.7742550


In [ ]:
# Método 3.
# Entrenamiento simple.

# Copiamos las variables para poder
# transformarlas en caso de ser necesario.
X_reg = X_train.copy()
Y_reg = Y_train.copy()
X_reg_val = X_val.copy()
X_reg_test = X_test.copy()

# Hiperparámetros.
params = {
    "objective": "reg:squarederror",
    "colsample_bytree": 0.3,
    "learning_rate": 0.1,
    "max_depth": 50,
    "min_child_weight" : 25,
    "min_split_loss": 0.3,
    "n_estimators": 100
    }

# Creamos el regresor con los hiperparámetros.
xg_reg_3 = xgb.XGBRegressor( **params )

# Entrenamos el modelo.
xg_reg_3.fit(X_reg, Y_reg, verbose = True)

# Probamos la regresión en el set de entrenamiento.
preds = xg_reg_3.predict(X_reg)

# Calculamos el error de entrenamiento.
rmse = np.sqrt(mean_squared_error(Y_reg, preds))
r2 = r2_score(Y_reg, preds)

print("Entrenamiento")
print(f"RMSE: {rmse:.3E}")
print(f"R^2:  {r2:.7f}")

Entrenamiento
RMSE: 3.559E+05
R^2:  0.9976006


In [ ]:
# Probamos la regresión en el set de validación.
preds = xg_reg_3.predict(X_reg_val)

# Calculamos el error de validación.
rmse = np.sqrt(mean_squared_error(Y_val, preds))
r2 = r2_score(Y_val, preds)

print("Validación")
print(f"RMSE: {rmse:.3E}")
print(f"R^2:  {r2:.7f}")

# Probamos la regresión en el set de prueba
# y guardamos el resultado para su evaluación final.
test_3 = xg_reg_3.predict(X_reg_test)

test.append(test_3)
reg_model.append(xg_reg_3)

Validación
RMSE: 6.121E+05
R^2:  0.9931037


In [ ]:
# Método 4.
# Entrenamiento simple con
# transformación logarítmica.

# Copiamos las variables para poder
# transformarlas en caso de ser necesario.
X_reg = X_train.copy()
Y_reg = Y_train.copy()
X_reg_val = X_val.copy()
X_reg_test = X_test.copy()

# Sacamos el logaritmo.
X_reg[["Poblacion", "PIB"]] = np.log(
    X_reg[["Poblacion", "PIB"]])
Y_reg = np.log(Y_reg)
X_reg_val[["Poblacion", "PIB"]] = np.log(
    X_reg_val[["Poblacion", "PIB"]])
X_reg_test[["Poblacion", "PIB"]] = np.log(
    X_reg_test[["Poblacion", "PIB"]])

# Hiperparámetros.
params = {
    "objective": "reg:squarederror",
    "colsample_bytree": 0.3,
    "learning_rate": 0.1,
    "max_depth": 50,
    "min_child_weight" : 25,
    "min_split_loss": 0.3,
    "n_estimators": 100
    }

# Creamos el regresor con los hiperparámetros.
xg_reg_4 = xgb.XGBRegressor( **params )

# Entrenamos el modelo.
xg_reg_4.fit(X_reg, Y_reg, verbose = True)

# Probamos la regresión en el set de entrenamiento.
preds = xg_reg_4.predict(X_reg)

# Quitamos el logaritmo para evaluar el error.
Y_reg = np.exp(Y_reg)
preds = np.exp(preds)

# Calculamos el error de entrenamiento.
rmse = np.sqrt(mean_squared_error(Y_reg, preds))
r2 = r2_score(Y_reg, preds)

print("Entrenamiento")
print(f"RMSE: {rmse:.3E}")
print(f"R^2:  {r2:.7f}")

Entrenamiento
RMSE: 6.619E+05
R^2:  0.9917004


In [ ]:
# Probamos la regresión en el set de validación.
preds = xg_reg_4.predict(X_reg_val)

# Quitamos el logaritmo para evaluar el error.
preds = np.exp(preds)

# Calculamos el error de validación.
rmse = np.sqrt(mean_squared_error(Y_val, preds))
r2 = r2_score(Y_val, preds)

print("Validación")
print(f"RMSE: {rmse:.3E}")
print(f"R^2:  {r2:.7f}")

# Probamos la regresión en el set de prueba
# y guardamos el resultado para su evaluación final.
test_4 = xg_reg_4.predict(X_reg_test)
test_4 = np.exp(test_4)

test.append(test_4)
reg_model.append(xg_reg_4)

Validación
RMSE: 8.104E+05
R^2:  0.9879101


In [ ]:
'''
# Método 5.
# Cross-Validation.

# Copiamos las variables para poder
# transformarlas en caso de ser necesario.
X_reg = X_train.copy()
Y_reg = Y_train.copy()
X_reg_val = X_val.copy()
X_reg_test = X_test.copy()

# Sacamos el logaritmo.
X_reg[["Poblacion", "PIB"]] = np.log(
    X_reg[["Poblacion", "PIB"]])
Y_reg = np.log(Y_reg)
X_reg_val[["Poblacion", "PIB"]] = np.log(
    X_reg_val[["Poblacion", "PIB"]])
X_reg_test[["Poblacion", "PIB"]] = np.log(
    X_reg_test[["Poblacion", "PIB"]])

# Hiperparámetros.
params = {
    "colsample_bytree": [ 0.3 ],
    "learning_rate": [ 0.1 ],
    "max_depth": [ 50 ],
    "min_child_weight" : [ 25 ],
    "min_split_loss": [ 0.3 ],
    "n_estimators": [ 100 ]
    }

# Creamos el regresor.
xg_reg = xgb.XGBRegressor(
    objective = "reg:squarederror"
    )

# Ajustador de hiperparámetros con Cross Validation.
grid_5 = GridSearchCV(xg_reg, params,
    n_jobs = -1, verbose = 2,
    scoring = "neg_mean_absolute_error")

# Entrenamos el modelo.
grid_5.fit(X_reg, Y_reg)

# Probamos la regresión en el set de entrenamiento.
preds = grid_5.predict(X_reg)

# Calculamos el error de entrenamiento.
rmse = np.sqrt(mean_squared_error(Y_reg, preds))
r2 = r2_score(Y_reg, preds)

print("Entrenamiento")
print(f"RMSE: {rmse:.3E}")
print(f"R^2:  {r2:.7f}")
'''

'\n# Método 5.\n# Cross-Validation.\n\n# Copiamos las variables para poder\n# transformarlas en caso de ser necesario.\nX_reg = X_train.copy()\nY_reg = Y_train.copy()\nX_reg_val = X_val.copy()\nX_reg_test = X_test.copy()\n\n# Sacamos el logaritmo.\nX_reg[["Poblacion", "PIB"]] = np.log(\n    X_reg[["Poblacion", "PIB"]])\nY_reg = np.log(Y_reg)\nX_reg_val[["Poblacion", "PIB"]] = np.log(\n    X_reg_val[["Poblacion", "PIB"]])\nX_reg_test[["Poblacion", "PIB"]] = np.log(\n    X_reg_test[["Poblacion", "PIB"]])\n\n# Hiperparámetros.\nparams = {\n    "colsample_bytree": [ 0.3 ],\n    "learning_rate": [ 0.1 ],\n    "max_depth": [ 50 ],\n    "min_child_weight" : [ 25 ],\n    "min_split_loss": [ 0.3 ],\n    "n_estimators": [ 100 ]\n    }\n\n# Creamos el regresor.\nxg_reg = xgb.XGBRegressor(\n    objective = "reg:squarederror"\n    )\n\n# Ajustador de hiperparámetros con Cross Validation.\ngrid_5 = GridSearchCV(xg_reg, params,\n    n_jobs = -1, verbose = 2,\n    scoring = "neg_mean_absolute_error")\

In [ ]:
'''
# Probamos la regresión en el set de validación.
preds = grid_5.predict(X_reg_val)

# Quitamos el logaritmo para evaluar el error.
preds = np.exp(preds)

# Calculamos el error de validación.
rmse = np.sqrt(mean_squared_error(Y_val, preds))
r2 = r2_score(Y_val, preds)

print("Validación")
print(f"RMSE: {rmse:.3E}")
print(f"R^2:  {r2:.7f}")

# Probamos la regresión en el set de prueba
# y guardamos el resultado para su evaluación final.
test_5 = grid_5.predict(X_reg_test)
test_5 = np.exp(test_5)

test.append(test_5)
reg_model.append(grid_5)
'''

'\n# Probamos la regresión en el set de validación.\npreds = grid_5.predict(X_reg_val)\n\n# Quitamos el logaritmo para evaluar el error.\npreds = np.exp(preds)\n\n# Calculamos el error de validación.\nrmse = np.sqrt(mean_squared_error(Y_val, preds))\nr2 = r2_score(Y_val, preds)\n\nprint("Validación")\nprint(f"RMSE: {rmse:.3E}")\nprint(f"R^2:  {r2:.7f}")\n\n# Probamos la regresión en el set de prueba\n# y guardamos el resultado para su evaluación final.\ntest_5 = grid_5.predict(X_reg_test)\ntest_5 = np.exp(test_5)\n\ntest.append(test_5)\nreg_model.append(grid_5)\n'

In [ ]:
# Método 6.
# Ajuste de hiperparámetros.

# Tarda bastante en ejecutar, usar
# como base para los hiperparámetros
# en otros métodos.
'''
# Copiamos las variables para poder
# transformarlas en caso de ser necesario.
X_reg = X_train.copy()
Y_reg = Y_train.copy()
X_reg_val = X_val.copy()
X_reg_test = X_test.copy()

# Sacamos el logaritmo.
X_reg[["Poblacion", "PIB"]] = np.log(
    X_reg[["Poblacion", "PIB"]])
Y_reg = np.log(Y_reg)
X_reg_val[["Poblacion", "PIB"]] = np.log(
    X_reg_val[["Poblacion", "PIB"]])
X_reg_test[["Poblacion", "PIB"]] = np.log(
    X_reg_test[["Poblacion", "PIB"]])

# Hiperparámetros
params = {
    "colsample_bytree": [ 0.3 ],
    "learning_rate": [ 0.1 ],
    "learning_rate": [ 0.1, 0.2, 0.3, 0.4, 0.5 ],
    "max_depth": [ 50 ],
    "min_child_weight" : [ 0, 10, 15, 20, 25 ],
    "min_split_loss": [ 0.0, 0.3, 0.6, 0.9, 1.2 ],
    #"alpha": [ 0, 3, 6, 9, 12 ],
    #"lambda": [ 0, 3, 6, 9, 12 ],
    "n_estimators": [ 100 ],
    }

# Creamos el regresor.
xg_reg = xgb.XGBRegressor(
    objective = "reg:squarederror")

# Ajustador de hiperparámetros con Cross Validation.
grid_6 = GridSearchCV(xg_reg, params,
    n_jobs = -1, cv = 2, verbose = 2,
    scoring = "neg_mean_absolute_error")

# Entrenamos el modelo.
grid_6.fit(X_reg, Y_reg)

# Probamos la regresión en el set de entrenamiento.
preds = grid_6.predict(X_reg)

# Calculamos el error de entrenamiento.
rmse = np.sqrt(mean_squared_error(Y_reg, preds))
r2 = r2_score(Y_reg, preds)

print("Entrenamiento")
print(f"RMSE: {rmse:.3E}")
print(f"R^2:  {r2:.7f}")
print("Mejores parámetros")
print(grid_6.best_params_)
'''

'\n# Copiamos las variables para poder\n# transformarlas en caso de ser necesario.\nX_reg = X_train.copy()\nY_reg = Y_train.copy()\nX_reg_val = X_val.copy()\nX_reg_test = X_test.copy()\n\n# Sacamos el logaritmo.\nX_reg[["Poblacion", "PIB"]] = np.log(\n    X_reg[["Poblacion", "PIB"]])\nY_reg = np.log(Y_reg)\nX_reg_val[["Poblacion", "PIB"]] = np.log(\n    X_reg_val[["Poblacion", "PIB"]])\nX_reg_test[["Poblacion", "PIB"]] = np.log(\n    X_reg_test[["Poblacion", "PIB"]])\n\n# Hiperparámetros\nparams = {\n    "colsample_bytree": [ 0.3 ],\n    "learning_rate": [ 0.1 ],\n    "learning_rate": [ 0.1, 0.2, 0.3, 0.4, 0.5 ],\n    "max_depth": [ 50 ],\n    "min_child_weight" : [ 0, 10, 15, 20, 25 ],\n    "min_split_loss": [ 0.0, 0.3, 0.6, 0.9, 1.2 ],\n    #"alpha": [ 0, 3, 6, 9, 12 ],\n    #"lambda": [ 0, 3, 6, 9, 12 ],\n    "n_estimators": [ 100 ],\n    }\n\n# Creamos el regresor.\nxg_reg = xgb.XGBRegressor(\n    objective = "reg:squarederror")\n\n# Ajustador de hiperparámetros con Cross Validatio

In [ ]:
'''
# Probamos la regresión en el set de validación.
preds = grid_6.predict(X_reg_val)

# Quitamos el logaritmo para evaluar el error.
preds = np.exp(preds)

# Calculamos el error de validación.
rmse = np.sqrt(mean_squared_error(Y_val, preds))
r2 = r2_score(Y_val, preds)

print("Validación")
print(f"RMSE: {rmse:.3E}")
print(f"R^2:  {r2:.7f}")

# Probamos la regresión en el set de prueba
# y guardamos el resultado para su evaluación final.
test_6 = grid_6.predict(X_reg_test)
test_6 = np.exp(test_6)

test.append(test_6)
reg_model.append(grid_6)
'''

'\n# Probamos la regresión en el set de validación.\npreds = grid_6.predict(X_reg_val)\n\n# Quitamos el logaritmo para evaluar el error.\npreds = np.exp(preds)\n\n# Calculamos el error de validación.\nrmse = np.sqrt(mean_squared_error(Y_val, preds))\nr2 = r2_score(Y_val, preds)\n\nprint("Validación")\nprint(f"RMSE: {rmse:.3E}")\nprint(f"R^2:  {r2:.7f}")\n\n# Probamos la regresión en el set de prueba\n# y guardamos el resultado para su evaluación final.\ntest_6 = grid_6.predict(X_reg_test)\ntest_6 = np.exp(test_6)\n\ntest.append(test_6)\nreg_model.append(grid_6)\n'

In [ ]:
i = 4

# Calculamos el error de prueba.
rmse = np.sqrt(mean_squared_error(Y_test, test[i - 1]))
r2 = r2_score(Y_test, test[i - 1])

print("Prueba")
print(f"Método {i}")
print(f"RMSE: {rmse:.3E}")
print(f"R^2:  {r2:.7f}")

# Guardamos el modelo entrenado.
dump(reg_model[i - 1], path_drive + "reg_model_consumo.joblib")

Prueba
Método 3
RMSE: 5.056E+05
R^2:  0.9952608


['/content/drive/MyDrive/Colab/Subsidio_electricidad/reg_model_consumo.joblib']